<IMG SRC="https://github.com/jacquesroy/byte-size-data-science/raw/master/images/Banner.png" ALT="BSDS Banner" WIDTH=1195 HEIGHT=200>

# Geocoding - Spatial objects
In this notebook we cover two things:
- Geocoding 
    - from an address to a (longitude, latitude)
    - From a (longitude, latititude) to an address
- Spatial objects beyond points

## Geocoding
Here are some geolocation service you might use in geopy:

| 1 | 2 | 3 | 4 | 5 | 6 | 7 | 8 | 9 | 10 |
|------|------|------|------|------|------|------|------|------|------|
| ArcGIS | AzureMaps | Baidu | BANFrance | Bing | DataBC | GeocodeEarth | GeocodeFarm | Geolake | GeoNames |
| GoogleV3 | HERE | IGNFrance | MapBox | OpenCage | OpenMapQuest | Nominatim | Pelias | Photon | PickPoint |
| LiveAddress | TomTom | What3Words | Yandex |  

In this example, I use **`Nominatim`** since I don't need to create any type of account.

See the documentation at: https://geopy.readthedocs.io/en/1.18.1/#nominatim

Under the cover, Nominatim uses OpenStreetMap that uses the WGS-84 coordinate system.

By default Folium uses EPSG3857 which appears to be the same WGS-84


### 046-Geocoding
Execute the next cell if you want to see the `Byte Size Data Science` youtube channel video

In [ ]:
from IPython.display import IFrame

IFrame(src="https://www.youtube.com/embed/ByckZOLx3HU?rel=0&amp;controls=0&amp;showinfo=0", width=560, height=315)


In [1]:
# DataBC: Service from British Columbia, Canada
# Nominatim: Open source geocoding from OpenStreetMap data
# Photon: open source geocoder built for OpenStreetMap data (from Komoot)
# MapBox: location data platform for mobile and web applications. (can get a free account)

geocoder_services = ['DataBC', 'Nominatim', 'Photon', 'MapBox']
svc = geocoder_services[1]

from geopy.geocoders import DataBC, Nominatim, Photon, MapBox

geolocator = None

if svc == 'DataBC':
    geolocator = DataBC() 
if svc == 'Nominatim':
    geolocator = Nominatim(user_agent="geocodingNotebook")
if svc == 'Photon':
    geolocator = Photon()
if svc == 'MapBox': 
    geolocator = MapBox("pk.eyJ1IjoiamFjcXVlc3IiLCJhIjoiY2ozdWF5OW4zMDBleTJxbnhvM2ozYnRjaCJ9.OoWWwKVXz10VGI2ToAVD3w")

In [2]:
type(geolocator)

geopy.geocoders.osm.Nominatim

### Geocoding results
Result of geocoding for: 17305 Northwest Corridor Ct Beaverton 97006
- DataBC: (53.913051, -122.7452849) # St-George, BC (the service is only for B.C. Canada
- Nominatim: (45.535417675371, -122.85443760727) # 265 feet from reverse location
- Photon: Timed out
- MapBox: (45.5365, -122.855143) # 0.15 feet from reverse location, matches my desired location

In [3]:
# The location I really wanted is at (45.5365886,-122.8551229)
# 17305 NW Corridor Ct Suite 100, Beaverton, Oregon 97006, United States

if svc == 'DataBC' :
    location = geolocator.geocode("17305 Northwest Corridor Ct Beaverton 97006", 
                                  exactly_one=True)
if svc == 'Nominatim' :
    location = geolocator.geocode("17305 Northwest Corridor Ct Beaverton 97006", 
                                  exactly_one=True, addressdetails=True, extratags=True)
if svc == 'Photon' :
    location = geolocator.geocode("17305 Northwest Corridor Ct Beaverton 97006", 
                                  exactly_one=True)
if svc == 'MapBox' :
    location = geolocator.geocode("17305 NW Corridor Ct, Beaverton, Oregon 97006, United States", 
                                  exactly_one=True)
                              
if (location == None) :
    print("Not found")
else :
    print(location.address)
    print((location.latitude, location.longitude))
    print(" ")
    print(location.raw)

GeocoderUnavailable: Service not available

In [ ]:
print(type(location))
dir(location)

In [ ]:
if svc != 'DataBC' :
    param = str(location.latitude) + " " + str(location.longitude)
    location2 = geolocator.reverse(param)

    print("Parameter passed: " + param)
    print(location2.address)
    print((location2.latitude, location2.longitude))
    print(' ')
    print(location2.raw)

In [ ]:
from geopy.distance import geodesic

p0=(45.5365886,-122.8551229, "Five Star Guitars (17305)") # the location of the address I really wanted (suite 100)
p1 = (location.latitude, location.longitude, "What I got")

if svc != 'DataBC' :
    p2 = (location2.latitude, location2.longitude, "What reverse returned (17235)")

    print("Distance between the two points: " + str(geodesic(p1[0:2], p2[0:2]).feet) + " feet")

In [ ]:
if svc != 'DataBC' :
    param = str(p0[0]) + " " + str(p0[1])
    location3 = geolocator.reverse(param)

    print("Parameter passed: " + param)
    print(location3.address)
    print((location3.latitude, location3.longitude))
    print(' ')
    print(location3.raw)

In [ ]:
# If you want to know what is available in the geodetic class:
# x = geodesic(p1[0:2], p2[0:2])
# dir(x)

## Show the difference on a map

In [7]:
# !conda install -c conda-forge folium=0.5.0 --yes
# !pip install folium==0.5.0
# I'm installing the latest version: 0.10.0
!pip install folium 2>&1 >foliumpip.out

import folium

In [ ]:
if svc == 'Nominatim' :
    point_list = [p0, p1, p2]
if svc == 'MapBox' :
    point_list = [p0, p1, p2]
if svc == 'DataBC' :
        point_list = [p1]

if svc == 'Nominatim' :
# bounding box
    b=[float(i) for i in location.raw['boundingbox']]
    bbox={
        "coordinates": [[[b[2], b[0]], [b[3], b[0]], 
                       [b[3], b[1]], [b[2], b[1]], 
                       [b[2], b[0]]]], 
        "type": "Polygon"
    }
    b2=[float(i) for i in location2.raw['boundingbox']]
    bbox2={
        "coordinates": [[[b2[2], b2[0]], [b2[3], b2[0]], 
                       [b2[3], b2[1]], [b2[2], b2[1]], 
                       [b2[2], b2[0]]]], 
        "type": "Polygon"
    }

loc_map = folium.Map(location=[location.latitude, location.longitude], crs='EPSG3857', zoom_start=17, width="80%", height="80%")
geo_objects = folium.map.FeatureGroup()

if svc == 'Nominatim' :
    # Add the bounding box of the original address
    geo_objects.add_child(folium.GeoJson(
            bbox,
            name='Original address',
            tooltip='Original address',
            overlay=True
        )
    )
    # Add the bounding box of the reverse address
    geo_objects.add_child(folium.GeoJson(
            bbox2,
            name='Reverse address',
            tooltip='Reverse address',
            overlay=True
        )
    )

for lat, lng, cmt in point_list:
    geo_objects.add_child(
        folium.CircleMarker(
            [lat, lng],
            radius=5, # define how big you want the circle markers to be
            color='yellow',
            fill=True,
            fill_color='blue',
            tooltip=cmt,
            fill_opacity=0.6
        )
)

loc_map.add_child(geo_objects)
loc_map

## Spatial objects
There are multiple types of spatial objects. Here are some of them in a well-known text representation.<br/>
(examples from https://en.wikipedia.org/wiki/Well-known_text_representation_of_geometry)

- Point: `POINT (30 10)`
- Linestring: `LINESTRING (30 10, 10 30, 40 40)` (line from point to point)
- Polygon: `POLYGON ((30 10, 40 40, 20 40, 10 20, 30 10))`
- Other: Multipoint, Multilinestring, Multipolygon, Geometry collection

In [4]:
# Redirecting the output to a file in case of problems
!pip install geopandas 2>&1 >pipgeopandas.txt

In [5]:
import urllib.request
import shutil
import geopandas as gp

place_file='https://github.com/jacquesroy/byte-size-data-science/raw/master/data/fourcities.json'

with urllib.request.urlopen(place_file) as response, open('fourcities.json', 'wb') as out_file:
    shutil.copyfileobj(response, out_file)

gdf_places = gp.read_file('fourcities.json')
gdf_places.head()

,NAME,INTPTLAT,INTPTLON,geometry
0,Schaumburg,42.028900,-88.083774,"POLYGON ((-88.15443 42.03188, -88.15371 42.031..."
1,Naperville,41.749173,-88.162019,"MULTIPOLYGON (((-88.13571 41.79292, -88.13530 ..."
2,La Grange,41.807214,-87.874146,"POLYGON ((-87.88943 41.81923, -87.88919 41.819..."
3,Chicago,41.837551,-87.681844,"POLYGON ((-87.94010 42.00079, -87.93993 42.000..."
4,Westchester,41.849196,-87.890619,"POLYGON ((-87.91896 41.84423, -87.91878 41.845..."


### Display on a map

In [9]:
type(gdf_places.iloc[1]['geometry'])

shapely.geometry.multipolygon.MultiPolygon

In [8]:

latlong = gdf_places[['INTPTLAT', 'INTPTLON']].mean()
il_map = folium.Map(location=[latlong[0], latlong[1]], zoom_start=10, width="80%", height="80%")

# Adding the city borders
for ix in range(gdf_places['NAME'].count()) :
    folium.GeoJson(
        gdf_places.iloc[ix]['geometry'],
        name=gdf_places.iloc[ix]['NAME'],
        tooltip=gdf_places.iloc[ix]['NAME']
    ).add_to(il_map)

# Add the long lat point for each city
for ix in range(gdf_places['NAME'].count()) :
        folium.CircleMarker(
            [gdf_places.iloc[ix]['INTPTLAT'].item(), gdf_places.iloc[ix]['INTPTLON'].item()],
            radius=5, # define how big you want the circle markers to be
            color='yellow',
            fill=True,
            fill_color='blue',
            tooltip=gdf_places.iloc[ix]['NAME'] + " ctr",
            fill_opacity=0.6
        ).add_to(il_map)
    
folium.LayerControl().add_to(il_map)
il_map

## Save cities info into Cloud Storage

In [ ]:
# Save the minimum info for later
!rm fourcities.json
gdf_places[['NAME','INTPTLAT', 'INTPTLON', 'geometry']].iloc[places_ix].to_file('fourcities.json', driver='GeoJSON')
!ls -l four*

In [ ]:
from ibm_botocore.client import Config
import ibm_boto3

# @hidden_cell
# The following code contains the credentials for a file in your IBM Cloud Object Storage.
# You might want to remove those credentials before you share your notebook.
credentials = {
    'IAM_SERVICE_ID': 'iam-ServiceId-65647f57-cd61-4f37-abd9-38b10084d177',
    'IBM_API_KEY_ID': '9OBEPHS0jj5q0FdEFWpF-US8qWWwiqFtRkeH6njgVaar',
    'ENDPOINT': 'https://s3-api.us-geo.objectstorage.service.networklayer.com',
    'IBM_AUTH_ENDPOINT': 'https://iam.ng.bluemix.net/oidc/token',
    'BUCKET': 'bscstesting-donotdelete-pr-paqxy5fmsmaykn',
    'FILE': 'BMWM3Coupe.zip'
}

cos = ibm_boto3.client(service_name='s3',
    ibm_api_key_id=credentials['IBM_API_KEY_ID'],
    ibm_service_instance_id=credentials['IAM_SERVICE_ID'],
    ibm_auth_endpoint=credentials['IBM_AUTH_ENDPOINT'],
    config=Config(signature_version='oauth'),
    endpoint_url=credentials['ENDPOINT'])

cos.upload_file(Filename='fourcities.json',Bucket=credentials['BUCKET'],Key='fourcities.json')